In [3]:
print("strawberry")


import torch
import numpy as np
from tqdm import tqdm
import scipy.stats
import argparse
import os
import shutil
import shlex
from collections import defaultdict
from tqdm import tqdm
import sys
from natsort import natsorted
import proplot as pplt
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from twaidata.torchdatasets.DirectoryParser3DMRIDataset import *
print("banana")


strawberry
banana


In [5]:
# get the 3d dataloader
mss3_ds = MSS3InterRaterDataset()
lbc_ds = LBCInterRaterDataset()
challenge_ds = WMHChallengeInterRaterDataset()

#WMHChallengeInterRaterDataset()#LBCInterRaterDataset()#MSS3InterRaterDataset()


In [9]:
def get_xs_ys_vents(ds, ds_name):
    xs3d_test = []
    ys3d_test = []
    pv_region_masks = []

    if ds_name == "mss3":
        r0_id = "wmhes"
        r1_id = "wmhmvh"
        ds_ir_folder = "MSS3_InterRaterData"
    elif ds_name == "lbc":
        r0_id = "wmh"
        r1_id = "wmh_flthresh"
        ds_ir_folder = "LBC_InterRaterData"
    elif ds_name == "challenge":
        r0_id = "wmho3"
        r1_id = "wmho4"
        ds_ir_folder = "WMHChallenge_InterRaterData"


    for (xs, ys, ind) in tqdm(ds):
        if ind in ['MSS3_ED_073_V1', 'MSS3_ED_075_V1', 'MSS3_ED_078_V1', 'MSS3_ED_079_V1']:
            print("found")
            continue
        if r0_id in ys.keys() and r1_id in ys.keys():
            try:
                x = torch.stack([xs['FLAIR'], xs['mask'], xs['T1']], dim=0)
                y = torch.stack([ys[r0_id], ys[r1_id]], dim=0)

                vent_distance_map = f"/home/s2208943/ipdis/data/preprocessed_data/{ds_ir_folder}/imgs/{ind}_T1_vent_distance.nii.gz"
                vent_distance_map = sitk.ReadImage(vent_distance_map)
                vent_distance_map = sitk.GetArrayFromImage(vent_distance_map)
            except:
                print(f"failed for {ind}")
                continue

            xs3d_test.append(x)
            ys3d_test.append(y)
            pv_region_masks.append(torch.from_numpy(vent_distance_map < 10).type(torch.float32))
            
    return xs3d_test, ys3d_test, pv_region_masks

In [10]:
xs_mss3, ys_mss3, pvs_mss3 = get_xs_ys_vents(mss3_ds, "mss3")

 90%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                    | 61/68 [00:16<00:01,  5.10it/s]

found
found


 94%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏           | 64/68 [00:17<00:00,  5.78it/s]

found
found


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 68/68 [00:17<00:00,  3.80it/s]


In [11]:
xs_lbc, ys_lbc, pvs_lbc = get_xs_ys_vents(lbc_ds, "lbc")

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:04<00:00,  2.55it/s]


In [19]:
xs_challenge, ys_challenge, pvs_challenge = get_xs_ys_vents(challenge_ds, "challenge")

 38%|█████████████████████████████████████████████████████████████████████████████                                                                                                                            | 23/60 [00:10<00:15,  2.39it/s]

failed for Singapore_52


 48%|█████████████████████████████████████████████████████████████████████████████████████████████████▏                                                                                                       | 29/60 [00:13<00:11,  2.60it/s]

failed for Singapore_58


 52%|███████████████████████████████████████████████████████████████████████████████████████████████████████▊                                                                                                 | 31/60 [00:13<00:10,  2.81it/s]

failed for Singapore_60


 57%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉                                                                                       | 34/60 [00:15<00:09,  2.66it/s]

failed for Singapore_63


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 60/60 [00:25<00:00,  2.33it/s]


In [20]:
mss3_mean_sum = [(a.cuda().sum() + b.cuda().sum()).item() / 2 for (a, b) in ys_mss3]
lbc_mean_sum = [(a.cuda().sum() + b.cuda().sum()).item() / 2 for (a, b) in ys_lbc]
challenge_mean_sum = [(a.cuda().sum() + b.cuda().sum()).item() / 2 for (a, b) in ys_challenge]

In [25]:
lbc_mean_sum = torch.Tensor(lbc_mean_sum)
challenge_mean_sum = torch.Tensor(challenge_mean_sum)
mss3_mean_sum = torch.Tensor(mss3_mean_sum)

In [31]:
lbc_mean_sum.max(), lbc_mean_sum.argmax()

(tensor(4882.5000), tensor(1))

In [32]:
(challenge_mean_sum - 4882.5).abs().min(), (challenge_mean_sum - 4882.5).abs().argmin()

(tensor(22.5000), tensor(27))

In [33]:
(mss3_mean_sum - 4882.5).abs().min(), (mss3_mean_sum - 4882.5).abs().argmin()

(tensor(11.5000), tensor(46))

In [ ]:
lbc_idx = 1
challenge_idx = 27
mss3_idx = 46